## Import Libraries

In [1]:
## import libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
!pip install wget
import wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=3e3752f88da978f9e851174c7f14c08d6f8c847be96e73eac3153a4b4f7c1e01
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


## Download the ACP Generation Code from github Repo

In [2]:
acp_synthesis = wget.download('https://raw.githubusercontent.com/mhdshl/ACP_Seq2Seq/main/generate/acp_synthesis.py')
# acp_feature_extraction = wget.download('https://raw.githubusercontent.com/mhdshl/ACP_Seq2Seq/main/utils/acp_feature_extraction.py')

### Import all functions from the ACP generation code

In [3]:
from acp_synthesis import *
# from acp_feature_extraction import *

## Import Datasets

In [4]:
acp240 = pd.read_csv('https://raw.githubusercontent.com/mhdshl/ACP_Seq2Seq/main/Data/acp240.txt', header=None)
acp740 = pd.read_csv('https://raw.githubusercontent.com/mhdshl/ACP_Seq2Seq/main/Data/acp740.txt', header=None)

## Convert the Sequences into Text and Combine the Sequences in One List

In [5]:
acp740_txt = seq_to_text(acp740)
acp240_txt = seq_to_text(acp240)
acp_txt = acp240_txt + '\n' + acp740_txt
aa2idx, idx2aa, aa_as_int, aa_dict = unique_chars(acp740_txt)
print(aa2idx)
print(idx2aa)
print(aa_as_int)
print(aa_dict)

{'\n': 0, 'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20}
['\n' 'A' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'K' 'L' 'M' 'N' 'P' 'Q' 'R' 'S' 'T'
 'V' 'W' 'Y']
[ 6 10 19 ... 16 13  1]
['\n', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']


## Generate Random Sequences without any ACP Sequence Generation Model

In [6]:
write_random_text(aa2idx, idx2aa, min_threshold=11, max_threshold=97, filename='random_from_func.txt', num_generate=150000)

## Create Dataset for Sequence Generation Model Training

In [7]:
seq_batch, aa_dataset, idx2aa, aa_dict = create_batch(acp_txt)
dataset = seq_batch.map(split_input_target)
dataset = create_dataset(acp_txt)

## Train and Load the Model

In [9]:
model = train_gen_model(dataset, len(aa_dict), embedding_dim= 256, rnn_units= 1024, batch_size=64, EPOCHS=100)

Epoch 1/100
6/6 - 6s - loss: 3.0998 - 6s/epoch - 918ms/step
Epoch 2/100
6/6 - 1s - loss: 2.9835 - 1s/epoch - 189ms/step
Epoch 3/100
6/6 - 1s - loss: 2.9088 - 992ms/epoch - 165ms/step
Epoch 4/100
6/6 - 1s - loss: 2.8672 - 866ms/epoch - 144ms/step
Epoch 5/100
6/6 - 1s - loss: 2.8542 - 786ms/epoch - 131ms/step
Epoch 6/100
6/6 - 1s - loss: 2.8352 - 830ms/epoch - 138ms/step
Epoch 7/100
6/6 - 4s - loss: 2.8285 - 4s/epoch - 599ms/step
Epoch 8/100
6/6 - 1s - loss: 2.8204 - 1s/epoch - 173ms/step
Epoch 9/100
6/6 - 1s - loss: 2.8070 - 810ms/epoch - 135ms/step
Epoch 10/100
6/6 - 1s - loss: 2.7965 - 1s/epoch - 206ms/step
Epoch 11/100
6/6 - 1s - loss: 2.7911 - 892ms/epoch - 149ms/step
Epoch 12/100
6/6 - 1s - loss: 2.7651 - 1s/epoch - 174ms/step
Epoch 13/100
6/6 - 1s - loss: 2.7543 - 876ms/epoch - 146ms/step
Epoch 14/100
6/6 - 4s - loss: 2.7452 - 4s/epoch - 673ms/step
Epoch 15/100
6/6 - 1s - loss: 2.7182 - 880ms/epoch - 147ms/step
Epoch 16/100
6/6 - 1s - loss: 2.6959 - 855ms/epoch - 143ms/step
Epoch 

## Generate ACP Sequences using the Trained Model

In [12]:
f = open('gen_from_model.txt', 'w')
f.write(generate_text(aa2idx,idx2aa, model,min_thres=11,max_thres=97, num_generate=150000, temperature=1.0))
f.close()